In [1]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")

import os
paths = [
    "/kaggle/input/resume-dataset/Resume/Resume.csv",
    "/root/.cache/kagglehub/datasets/snehaanbhawal/resume-dataset/versions/1/Resume/Resume.csv"
]

for p in paths:
    try:
        df = pd.read_csv(p, encoding="latin-1")
        print("Successfully loaded:", p)
        break
    except:
        pass


Using Colab cache for faster access to the 'resume-dataset' dataset.
Successfully loaded: /kaggle/input/resume-dataset/Resume/Resume.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.listdir('/content/drive/MyDrive'))

Mounted at /content/drive
['aurora.zip', 'interested jobs.gsheet', 'Critical Analysis Content Summary #3.gdoc', 'Items to sell.gsheet', 'Colab Notebooks', 'ONET']


In [3]:
import os
print(os.listdir('/content/drive/MyDrive/ONET'))

['Scales Reference.txt', 'Task Categories.txt', 'IWA Reference.txt', 'Education, Training, and Experience Categories.txt', 'Occupation Data.txt', 'Work Values.txt', 'Abilities to Work Activities.txt', 'Level Scale Anchors.txt', 'Skills to Work Activities.txt', 'UNSPSC Reference.txt', 'Basic Interests to RIASEC.txt', 'Interests Illustrative Occupations.txt', 'RIASEC Keywords.txt', 'Interests Illustrative Activities.txt', 'Interests.txt', 'Related Occupations.txt', 'Job Zone Reference.txt', 'Tools Used.txt', 'Abilities to Work Context.txt', 'Content Model Reference.txt', 'Survey Booklet Locations.txt', 'Skills to Work Context.txt', 'Work Context Categories.txt', 'Technology Skills.txt', 'Read Me.txt', 'Abilities.txt', 'Alternate Titles.txt', 'DWA Reference.txt', 'Education, Training, and Experience.txt', 'Emerging Tasks.txt', 'Job Zones.txt', 'Knowledge.txt', 'Occupation Level Metadata.txt', 'Skills.txt', 'Task Ratings.txt', 'Task Statements.txt', 'Tasks to DWAs.txt', 'Work Activities.tx

In [4]:
import pandas as pd

skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
abilities = pd.read_csv('/content/drive/MyDrive/ONET/Abilities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
occupation = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')

In [5]:
import pandas as pd

# Loading key ONET files
occ = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')

# Group aggregation (join multiple rows per occupation)
occ_skills = skills.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()
occ_tasks = tasks.groupby('O*NET-SOC Code')['Task'].apply(list).reset_index()
occ_activities = activities.groupby('O*NET-SOC Code')['Element Name'].apply(list).reset_index()

# Merge them all
occupation_profile = occ[['O*NET-SOC Code', 'Title', 'Description']] \
    .merge(occ_skills, on='O*NET-SOC Code', how='left') \
    .merge(occ_tasks, on='O*NET-SOC Code', how='left') \
    .merge(occ_activities, on='O*NET-SOC Code', how='left')

occupation_profile.head()

,O*NET-SOC Code,Title,Description,Element Name_x,Task,Element Name_y
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,"[Reading Comprehension, Reading Comprehension,...",[Direct or coordinate an organization's financ...,"[Getting Information, Getting Information, Mon..."
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, sh...","[Reading Comprehension, Reading Comprehension,...",[Monitor and evaluate effectiveness of sustain...,"[Getting Information, Getting Information, Mon..."
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ...","[Reading Comprehension, Reading Comprehension,...","[Review financial statements, sales or activit...","[Getting Information, Getting Information, Mon..."
3,11-1031.00,Legislators,"Develop, introduce, or enact laws and statutes...",NaN,[Analyze and understand the local and national...,NaN
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...","[Reading Comprehension, Reading Comprehension,...",[Plan and prepare advertising and promotional ...,"[Getting Information, Getting Information, Mon..."


In [6]:
!pip install sentence-transformers

In [7]:
print(occupation_profile.columns)
occupation_profile = occupation_profile.rename(columns={
    'Element Name_x': 'Skills',
    'Task': 'Tasks',
    'Element Name_y': 'Work_Activities'
})
for col in ['Skills', 'Tasks', 'Work_Activities']:
    occupation_profile[col] = occupation_profile[col].apply(
        lambda x: list(set(x)) if isinstance(x, list) else x
    )
def combine_text(row):
    text = f"{row['Title']} {row['Description']} "
    if isinstance(row['Skills'], list):
        text += ' '.join(row['Skills'])
    if isinstance(row['Tasks'], list):
        text += ' '.join(row['Tasks'])
    if isinstance(row['Work_Activities'], list):
        text += ' '.join(row['Work_Activities'])
    return text
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
occupation_profile['combined_text'] = occupation_profile.apply(combine_text, axis=1)
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))


Index(['O*NET-SOC Code', 'Title', 'Description', 'Element Name_x', 'Task',
       'Element Name_y'],
      dtype='object')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def explain_match(resume_skills, job_skills):
    if isinstance(job_skills, list):
        return list(set([s.lower() for s in job_skills]) & set(resume_skills))
    return []

In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Resume dataset
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print(resumes.columns)  # Check actual column names

# Example: if resume text column is named 'Resume', otherwise adjust:
resume_text_column = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET dataset
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print(tasks.columns)  # Check what columns exist

Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')


In [ ]:
# Install required libraries
!pip install sentence-transformers --quiet

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. Load Datasets

# Resume dataset (Kaggle)
resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
print("Resume columns:", resumes.columns)

# Assume first column contains resume text
resume_text_col = 'Resume' if 'Resume' in resumes.columns else resumes.columns[0]

# O*NET occupation dataset (adjust path in Colab)
# Combine relevant fields later into 'combined_text'
onet_path = '/content/drive/MyDrive/ONET/Task Statements.txt'
tasks = pd.read_csv(onet_path, sep='\t', encoding='latin-1')
print("O*NET columns:", tasks.columns)

# For demonstration, let's load Skills.txt as well
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')

# Merge or create an occupation profile (simplified)
occupation_profile = skills.copy()
occupation_profile['combined_text'] = occupation_profile['Element Name'].astype(str)  # Use 'Element Name' or combine multiple columns



model = SentenceTransformer('all-MiniLM-L6-v2')


# Resumes
resumes['embedding'] = resumes[resume_text_col].astype(str).apply(lambda x: model.encode(x))

# Occupations
occupation_profile['embedding'] = occupation_profile['combined_text'].astype(str).apply(lambda x: model.encode(x))


def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    top_jobs = occupation_titles.iloc[top_idx]
    top_scores = sims[top_idx]
    return pd.DataFrame({'Job Title': top_jobs, 'Similarity': top_scores})

# Example: Recommend for first resume
first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Element Name'],
                                 top_k=5)

print("Top 5 job recommendations for first resume:")
print(recommendations)

Resume columns: Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
O*NET columns: Index(['O*NET-SOC Code', 'Task ID', 'Task', 'Task Type',
       'Incumbents Responding', 'Date', 'Domain Source'],
      dtype='object')
Top 5 job recommendations for first resume:
             Job Title  Similarity
50     Troubleshooting    0.211162
62560  Troubleshooting    0.211162
38691  Troubleshooting    0.211162
29241  Troubleshooting    0.211162
58920  Troubleshooting    0.211162


In [ ]:

!pip install sentence-transformers --quiet
!pip install umap-learn --quiet

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import re
import umap

resume_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resumes = pd.read_csv(resume_path)
resume_text_col = 'Resume_str'

# O*NET datasets
occupation_data = pd.read_csv('/content/drive/MyDrive/ONET/Occupation Data.txt', sep='\t', encoding='latin-1')
skills = pd.read_csv('/content/drive/MyDrive/ONET/Skills.txt', sep='\t', encoding='latin-1')
work_activities = pd.read_csv('/content/drive/MyDrive/ONET/Work Activities.txt', sep='\t', encoding='latin-1')
tasks = pd.read_csv('/content/drive/MyDrive/ONET/Task Statements.txt', sep='\t', encoding='latin-1')
knowledge = pd.read_csv('/content/drive/MyDrive/ONET/Knowledge.txt', sep='\t', encoding='latin-1')
work_context = pd.read_csv('/content/drive/MyDrive/ONET/Work Context.txt', sep='\t', encoding='latin-1')

def clean_resume(text):
    text = re.sub(r'<.*?>', ' ', str(text))  # remove HTML
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

resumes['cleaned_resume'] = resumes[resume_text_col].apply(clean_resume)


def aggregate_text(df, group_col='O*NET-SOC Code', text_col='Element Name', new_col_name=None):
    agg = df.groupby(group_col)[text_col].apply(lambda x: ' '.join(x.astype(str))).reset_index()
    if new_col_name:
        agg = agg.rename(columns={text_col: new_col_name})
    return agg

skills_agg = aggregate_text(skills, text_col='Element Name', new_col_name='Skills')
work_agg = aggregate_text(work_activities, text_col='Element Name', new_col_name='Work_Activities')
tasks_agg = aggregate_text(tasks, text_col='Task', new_col_name='Tasks')
knowledge_agg = aggregate_text(knowledge, text_col='Element Name', new_col_name='Knowledge')
context_agg = aggregate_text(work_context, text_col='Element Name', new_col_name='Work_Context')

occupation_profile = occupation_data.merge(skills_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(work_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(tasks_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(knowledge_agg, on='O*NET-SOC Code', how='left')
occupation_profile = occupation_profile.merge(context_agg, on='O*NET-SOC Code', how='left')

occupation_profile['combined_text'] = (
    occupation_profile['Title'].astype(str) + ' ' +
    occupation_profile['Description'].astype(str) + ' ' +
    3*occupation_profile['Skills'].astype(str) + ' ' +       # weight skills higher
    2*occupation_profile['Tasks'].astype(str) + ' ' +       # weight tasks moderately
    occupation_profile['Work_Activities'].astype(str) + ' ' +
    occupation_profile['Knowledge'].astype(str) + ' ' +
    occupation_profile['Work_Context'].astype(str)

model = SentenceTransformer('all-mpnet-base-v2')

print("Generating occupation embeddings...")
occupation_profile['embedding'] = occupation_profile['combined_text'].apply(lambda x: model.encode(str(x)))

print("Generating resume embeddings...")
resumes['embedding'] = resumes['cleaned_resume'].apply(lambda x: model.encode(str(x)))

embeddings_matrix = list(occupation_profile['embedding'])
umap_model = umap.UMAP(n_components=256, random_state=42)
occupation_profile['embedding_reduced'] = list(umap_model.fit_transform(embeddings_matrix))



def recommend_jobs(resume_embedding, occupation_embeddings, occupation_titles, top_k=5):
    sims = cosine_similarity([resume_embedding], list(occupation_embeddings))[0]
    top_idx = sims.argsort()[::-1][:top_k]
    return pd.DataFrame({'Job Title': occupation_titles.iloc[top_idx], 'Similarity': sims[top_idx]})

first_resume_embedding = resumes['embedding'][0]
recommendations = recommend_jobs(first_resume_embedding,
                                 occupation_profile['embedding'],
                                 occupation_profile['Title'],
                                 top_k=5)
print("Top 5 job recommendations for first resume:")
print(recommendations)

all_recommendations = []
for idx, row in resumes.iterrows():
    recs = recommend_jobs(row['embedding'], occupation_profile['embedding'], occupation_profile['Title'], top_k=5)
    recs['Resume_ID'] = row['ID']
    all_recommendations.append(recs)

final_recs = pd.concat(all_recommendations, ignore_index=True)
final_recs.to_csv('resume_job_recommendations_final.csv', index=False)
print("Saved all top 5 job recommendations to 'resume_job_recommendations_final.csv'.")


import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(8, 5))
sns.histplot(final_recs['Similarity'], bins=20, kde=True)
plt.title("Distribution of Job Recommendation Similarity Scores")
plt.xlabel("Similarity Score")
plt.ylabel("Frequency")
plt.show()


top_jobs = final_recs['Job Title'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_jobs.values, y=top_jobs.index)
plt.title("Top 10 Most Frequently Recommended Job Titles")
plt.xlabel("Count")
plt.ylabel("Job Title")
plt.show()

# Scatter plot: Resume index vs. similarity score
plt.figure(figsize=(10, 6))
plt.scatter(final_recs['Resume_ID'], final_recs['Similarity'], alpha=0.7)
plt.title("Similarity Score Distribution per Resume")
plt.xlabel("Resume ID")
plt.ylabel("Similarity")
plt.show()

# Heatmap of top job titles vs average similarity
top_jobs_sim = final_recs.groupby('Job Title')['Similarity'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.heatmap(top_jobs_sim.to_frame(), annot=True, fmt=".3f", cmap="rocket")
plt.title("Average Similarity of Top 10 Recommended Job Titles")
plt.ylabel("Job Title")
plt.show()

# Save visual-ready summary
summary_stats = final_recs.groupby('Job Title')['Similarity'].agg(['count', 'mean', 'max']).sort_values(by='mean', ascending=False)
summary_stats.to_csv('job_similarity_summary.csv')
print("Visualization data saved to 'job_similarity_summary.csv'.")

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating occupation embeddings...
